In [1]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

import os

import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

import gphelper as gp
import ImageTools as it
import DataTools as dt

import importlib

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
X = np.linspace(0.001, 5, 500)[:, np.newaxis]
h_model = gp.GPHelper(restore_file='./gp-models/gp_h.json')
y_gp = 10**h_model.sample(X)

plt.title('H SBP')
plt.ylabel('$I/I_e$')
plt.xlabel('$R/R_e$')
plt.plot(X, y_gp)
plt.show()

/home/ryanhausen/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:339: RuntimeWarning: covariance is not positive-semidefinite.
  y_samples = rng.multivariate_normal(y_mean, y_cov, n_samples).T


In [9]:
def sbp_to_int(x, y):
    delta_x = np.diff(x)[0]/2
    xs = [np.pi*((_x+delta_x)**2-(_x-delta_x)**2) for _x in x.flatten()]
    xs = np.array(xs)
    
    weighted_y = xs * y
    I_tot = weighted_y.sum()
    int_I = np.cumsum(weighted_y/I_tot)
    return int_I, I_tot, xs

def img_sbp_to_int(x, y, Itot=None):
    I_tot = Itot if Itot else y.sum()
    I_int = np.cumsum(y/I_tot)
    
    return I_int

def int_to_sbp(x, y, y_tot):
    return (np.array([0] + np.diff(y).tolist())*y_tot)/x

def eta_line(x, difference_at_1):
    def delta_l(x, diff):
        return diff - diff * (1-x)**2
    
    def delta_r(x, diff):
        return diff - diff * ((x-1)/(x.max()-1))**2
    
    eta = np.zeros_like(x)
    eta[x<=1] = delta_l(x[x<=1], difference_at_1)
    eta[x>1] = delta_r(x[x>1], difference_at_1)

    return eta

# https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2.html
def sersic(x):
    def b(n):
        return 1.9992*n-0.3271

    def I(r):
        """Assuming that I_e=1.0 and that R_e=1.0"""
        n = 4
        return np.exp(-b(n) * (np.power(r, 1/n)  - 1))

    return I(x)

def get_random_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)
    
    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources

def make_nonnegative(img):
    epsilon = 0 if len(img[img<0])==0 else img[img>0].min()
    return img + abs(img.min()) + epsilon

In [ ]:
importlib.reload(it)

y_gp_int, I_tot, wx = sbp_to_int(X.flatten(), y_gp.flatten())
diff_at_1 = 0.5-y_gp_int[np.square(X-1).argmin()]
y_gp_eta = y_gp_int + eta_line(X.flatten(), diff_at_1)

sources = []
for img, segmap, img_id in get_random_spheroids(num=1):
    img = make_nonnegative(img)
    rs, fs, ie, re, Itot = iterative_profile_expansion(img, segmap, img_id)
    sources.append((rs, fs, re, ie, Itot))

X_sersic = np.linspace(0.001, 50, 5000)[:, np.newaxis]
y_sersic = sersic(X_sersic)
y_sersic_int, _, _ = sbp_to_int(X_sersic.flatten(), y_sersic.flatten())

plt.figure(figsize=(15, 10))
plt.title('Measured SBP')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$')
plt.xlim((0,5))

plt.semilogy(X, y_gp, label='GP')
plt.semilogy(X_sersic, y_sersic, label='Sersic')

for x, y, r, i, itot in sources:
    plt.semilogy(x/r, y/i, '--', alpha=0.3)
plt.legend()
    
plt.figure(figsize=(15, 10))
plt.title('Measured Integrations')
plt.xlabel('$R/R_e$')
plt.xlim((0,5))

plt.plot(X, y_gp_int, label='GP')
plt.plot(X, y_gp_eta, label='GP+eta')
plt.plot(X_sersic, y_sersic_int, label='Sersic')

for x, y, r, i, itot in sources:
    s_int = img_sbp_to_int(x.flatten(), y, Itot=itot)
    plt.plot(x/r, s_int, '--', alpha=0.5)
m = max([max(s[0]) for s in sources])
plt.legend()
    
plt.show()
    
plt.figure(figsize=(15, 10))
plt.title('Measured Integrations')
plt.xlabel('$R/R_e$')
plt.xlim((0,5))

plt.plot(X, y_gp_int, label='GP')
plt.plot(X, y_gp_eta, label='GP+eta')
plt.plot(X_sersic, y_sersic_int, label='Sersic')

plt.ylim(0,1)
for x, y, r, i, itot in sources:
    s_int = img_sbp_to_int(x.flatten(), y, Itot=itot)
    plt.plot(x/r, s_int, '--', alpha=0.5)
m = max([max(s[0]) for s in sources])
plt.legend()

plt.show()

In [ ]:
y_mean, y_std = h_model.predict(X, return_std=True)
y_pls_std = 10**(y_mean + y_std)
y_min_std = 10**(y_mean - y_std)
y_mean =  10 **y_mean

restored_gp_eta = int_to_sbp(wx, y_gp_eta, I_tot)
restored_gp = int_to_sbp(wx, y_gp_int.flatten(), I_tot)

plt.figure(figsize=(15, 10))
plt.title('Restored SBP')
plt.semilogy(X, y_gp, label='GP')
plt.semilogy(X, restored_gp_eta, label='GP+eta')
plt.semilogy(X_sersic, y_sersic, label='Sersic', color='purple')
plt.semilogy(X, y_mean, label='GP $\mu$', color='r')
plt.fill_between(X.flatten(), y_min_std, y_pls_std, label='GP $\pm\sigma$', alpha=0.2, color='r')
plt.xlim(0,5)
plt.legend()
plt.show()

In [5]:
a = np.arange(9).reshape(3,3)
np.pad(a, 3, 'constant')


array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 2, 0, 0, 0],
       [0, 0, 0, 3, 4, 5, 0, 0, 0],
       [0, 0, 0, 6, 7, 8, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [43]:
def get_re(img, src_map, rs):
    Itot = img[src_map].sum()
    
    rs = rs[src_map]
    fs = img[src_map]

    sorted_rs = np.argsort(rs)
    int_fs = np.cumsum(fs[sorted_rs]/Itot)
    re_idx = np.square(int_fs-0.5).argmin()

    rs = rs[sorted_rs]
    fs = fs[sorted_rs]

    return rs[re_idx]

def within_tolerance(_re, re, tolerance):
    return np.sqrt(np.square(_re-re)) <= tolerance 

def iterative_profile_expansion(img, segmap, img_id, with_graph=False, print_out=True, tolerance=0.0):
    # strip other sources from image
    if with_graph:
        plt.figure()
        plt.title('Original')
        plt.imshow(img, cmap='gray')
        plt.show()
    
    noise_bank = img[segmap==0].flatten()
    other_sources = np.logical_and(segmap!=0, segmap!=img_id)
    if other_sources.sum() > 0:
        np.place(img, other_sources, noise_bank)
        np.place(segmap, other_sources, [0])
    
    img = make_nonnegative(img)
    img -= np.sqrt(np.mean(np.square(noise_bank)))
    img[img<=0] = 1e-6
    img_bank = img[segmap==0].flatten()
    
    if with_graph:
        plt.figure()
        plt.title('Sources Removed/Noise Subtracted/Nonnegative')
        plt.imshow(img, cmap='gray')
        plt.show()

        plt.figure()
        plt.title('Transformed Original Segmap')
        plt.imshow(img, cmap='gray')
        plt.imshow(segmap==img_id, cmap='Blues', alpha=0.2)
        plt.show()
    
    
    src_map = segmap==img_id
    cx, cy = it.img_center(img, src_map)
    center = np.zeros_like(src_map)
    center[int(cy), int(cx)] = 100
    
    _rs, _fs, i, r = it.rs_fs_ie_re(img, src_map)
    if with_graph:
        f, a = plt.subplots(3,1)
        a[0].imshow(img, cmap='gray')
        a[0].imshow(src_map, cmap='Blues', alpha=0.2)
        
        a[1].semilogy(_rs/r, _fs/i)
        a[2].plot(_rs/r, img_sbp_to_int(_rs/r, _fs))

        plt.show()    
    
    
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    rs = np.sqrt(np.square(cx-xs) + np.square(cy-ys))
    re = r
    _re = 0
    
    max_re = 20.0
    if print_out:
        print(f'Initial re:{re}')
    while within_tolerance(re, _re, tolerance)==False:
        if re > max_re:
            raise Exception('Max Re Exceeded')
        
        src_map = np.logical_or(src_map, (rs<=(5*re)))
        
        _rs, _fs, i, r = it.rs_fs_ie_re(img, src_map)
        if with_graph:
            f, a = plt.subplots(3,1)
            a[0].imshow(img, cmap='gray')
            a[0].imshow(src_map, cmap='Blues', alpha=0.2)

            a[1].semilogy(_rs/r, _fs/i)
            a[2].plot(_rs/r, img_sbp_to_int(_rs/r, _fs))
            plt.show()   
        
        if np.any(np.all(src_map, axis=1)):
            if print_out:
                print('Padding Image')
            # the src_map taken the entire image
            pad_amt = int(5 * re)
            img = np.pad(img, pad_amt, 'constant')
            np.place(img, img==0, noise_bank)
            src_map = np.pad(src_map, pad_amt, 'constant')
            cx, cy = cx+pad_amt, cy+pad_amt
            center = np.zeros_like(src_map)
            center[int(cy), int(cx)] = 100
            xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
            rs = np.sqrt(np.square(cx-xs) + np.square(cy-ys))

        
        
        _re = re
        re = r
        if print_out:
            print(f'Found re:{re}')
    rs, fs, ie, re = it.rs_fs_ie_re(img, src_map)
    
    return rs, fs, ie, re, img[src_map].sum()

In [48]:
img, segmap, img_id = get_random_spheroids(num=1)[0]

iterative_profile_expansion(make_nonnegative(img), segmap, img_id, with_graph=True, tolerance=0.1)
iterative_profile_expansion(make_nonnegative(img), segmap, img_id, with_graph=True, tolerance=0.0)

Initial re:4.326698304779454
Found re:4.734465426583245
Found re:4.769817240345057
Initial re:4.291629297169231
Found re:4.65610704129497
Found re:4.65097777243092
Found re:4.65097777243092


(array([  0.44274246,   0.73473977,   0.77553263, ...,  23.56025652,
         23.56815607,  23.59932228]),
 array([  5.37010550e-01,   5.00970483e-01,   5.20060420e-01, ...,
          8.58926028e-03,   9.99999997e-07,   9.99999997e-07], dtype=float32),
 0.14596736,
 4.6509777724309203,
 35.444244)

In [36]:
sources = get_random_spheroids(num=-1)
converged = 0
for img, segmap, img_id in sources:
    try:
        iterative_profile_expansion(img, segmap, img_id, print_out=False)
    except Exception as e:
        continue
        
    converged += 1

print(f'Covverged:{converged} Total Sources:{len(sources)} {converged/len(sources)}%')
print(converged/len(sources))

Covverged:41 Total Sources:500 0.082%
0.082


In [53]:
tolerances = np.linspace(0, 0.5, 100)

results = []
for t in tolerances:
    converged = 0
    for img, segmap, img_id in sources:
        try:
            iterative_profile_expansion(img, segmap, img_id, print_out=False, tolerance=t)
        except Exception as e:
            continue

        converged += 1
        
    results.append(converged/len(sources))
    
plt.plot(tolerances, results, '.')
plt.show()